In [1]:
import pandas as pd
import numpy as np 
import altair as alt 

---

## Producer Price: Nitrogenous Fertilizer Manufacturing in USA

**Data**
<br>U.S. Bureau of Labor Statistics, Producer Price Index by Industry: Nitrogenous Fertilizer Manufacturing [PCU325311325311], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/PCU325311325311, February 25, 2025.

In [2]:
df = pd.read_csv('PCU325311325311_PC1.csv')
df['observation_date'] = pd.to_datetime(df['observation_date'])
df = df.dropna()
df = df.rename(columns={"observation_date": "Date", "PCU325311325311_PC1": "Change"})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 556 entries, 0 to 577
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    556 non-null    datetime64[ns]
 1   Change  556 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 13.0 KB


In [3]:
df = df.loc[df['Date'] >= pd.Timestamp('2000-01-01 00:00:00')].reset_index(drop=True)
df.head()

,Date,Change
0,2000-01-01,5.60201
1,2000-02-01,9.18964
2,2000-03-01,10.49587
3,2000-04-01,10.92851
4,2000-05-01,13.28848


In [4]:
threshold = 0
predicate = alt.datum.Change > threshold
color = alt.when(predicate).then(alt.value("#D35400")).otherwise(alt.value("steelblue"))

bars = alt.Chart(df, title=alt.TitleParams('USA Producer Price: Nitrogenous Fertilizer Manufacturing', anchor='middle')).mark_bar().encode(
    alt.X("Date:T", title=''),
    alt.Y("Change:Q", title='Change from year ago, %'),
    color=color,
)

(bars)

alt.Chart(...)

---

## Fertiliser Production in EU

**Data**
<br>Fertiliser production, retrieved from Directorate-General for Agriculture and Rural Development; https://agridata.ec.europa.eu/extensions/DashboardFertiliser/FertiliserProduction.html; Access: February 25, 2025.

In [5]:
df1 = pd.read_excel('9d3ce135-600e-4531-96cf-3c1434101a26.xlsx')
df1 = df1.dropna()
df1['Year'] = df1['Year'].astype('str')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Year          413 non-null    object
 1   Member State  413 non-null    object
 2   Quantity      413 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.8+ KB


In [6]:
df1 = df1.loc[df1['Year'] >= '2019'].reset_index(drop=True)
df1.head()

,Year,Member State,Quantity
0,2019,Austria,519693499
1,2019,Belgium,4631500519
2,2019,Bulgaria,2927608
3,2019,Croatia,1015104000
4,2019,Czechia,552875000


In [7]:
alt.Chart(df1, title=alt.TitleParams('Fertiliser Production in EU', anchor='middle')).mark_bar().encode(
    alt.X('sum(Quantity)', title='Fertiliser production, kg.'),
    alt.Y('Year',title=''),
    color=alt.Color('Member State', legend=alt.Legend(columns=2)).scale(scheme="tableau20") 
)

alt.Chart(...)

---

## Fertiliser Prices in EU

**Data**
<br>Fertiliser prices, retrieved from Directorate-General for Agriculture and Rural Development; https://agridata.ec.europa.eu/extensions/DashboardFertiliser/FertiliserPrices.html; Access: February 25, 2025.

In [8]:
df2 = pd.read_excel('2f560f9f-03d3-422f-b2f2-022c02d7b107.xlsx')
df2 = df2.dropna()
df2['Month_Year'] = pd.to_datetime(df2['Month_Year'])
df2 = df2.rename(columns={"Month_Year": "Date", "Price (€/tonne)": "Price"})
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     219 non-null    datetime64[ns]
 1   Product  219 non-null    object        
 2   Price    219 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 5.3+ KB


C:\Users\MCB User\AppData\Local\Temp\ipykernel_2860\2243489525.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['Month_Year'] = pd.to_datetime(df2['Month_Year'])


In [9]:
df2 = df2.loc[df2['Date'] >= pd.Timestamp('2019-01-01 00:00:00')].reset_index(drop=True)
df2.head()

,Date,Product,Price
0,2019-01-01,K (Potash),297
1,2019-01-01,N (Nitrogen),263
2,2019-01-01,P (Phosphorus),423
3,2019-02-01,K (Potash),295
4,2019-02-01,N (Nitrogen),249


In [11]:
chart = alt.Chart(df2, title=alt.TitleParams('Fertiliser Prices in EU', anchor='middle')).encode(
    alt.Color("Product").scale(scheme="dark2").legend(None)
)

line = chart.mark_line().encode(
    alt.X("Date:T", title=''),
    alt.Y("Price:Q", title='Price, €/tonne')
)

label = chart.encode(
    x='max(Date):T',
    y=alt.Y('Price:Q').aggregate(argmax='Date'),
    text='Product'
)

text = label.mark_text(align='left', dx=5)

circle = label.mark_circle()

line + circle + text

alt.LayerChart(...)

---